In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import gc
from scipy.fft import dct, idct
from scipy.stats import skew, kurtosis, entropy
from sklearn.utils.class_weight import compute_class_weight

# Fourier Decomposition Method (FDM)
def fdm(X, fs, fc):
    N = X.shape[0]
    fc = np.sort(fc)

    if fc[0] != 0:
        fc = np.hstack((0, fc))
    if fc[-1] != fs / 2:
        fc = np.hstack((fc, fs / 2))

    dct_type = 2
    K = np.round(2 * N * fc / fs).astype(int)
    Hk = np.zeros((N, 1, len(K) - 1))

    for i in range(len(K) - 1):
        Hk[K[i]:K[i + 1], :, i] = 1

    Xk = dct(X, type=dct_type, n=N, axis=0, norm='ortho')
    Yk = np.einsum('ij,ijk->ijk', Xk, Hk)
    Y = idct(Yk, type=dct_type, n=N, axis=0, norm='ortho')

    return np.squeeze(Y) if X.ndim == 1 else Y

# Feature Extraction
def extract_features(X):
    mean_amplitude = np.mean(X, axis=1, keepdims=True)
    variance = np.var(X, axis=1, keepdims=True)
    skewness = skew(X, axis=1, keepdims=True)
    kurt = kurtosis(X, axis=1, keepdims=True)
    
    X_normalized = np.abs(X) / np.sum(np.abs(X), axis=1, keepdims=True)
    signal_entropy = np.apply_along_axis(entropy, axis=1, arr=X_normalized).reshape(-1, 1)
    
    return np.hstack([mean_amplitude, variance, skewness, kurt, signal_entropy])

# Preprocess Data
def preprocess_data(df, label_column='Label', normalize=True):
    labels = df[label_column].values
    features = df.drop(columns=[label_column]).values
    features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
    additional_features = extract_features(features)
    features = np.hstack([features, additional_features])
    if normalize:
        features = features / np.max(np.abs(features), axis=1, keepdims=True)
    return features, labels

# Autoencoder Model
def build_autoencoder(input_shape):
    inputs = Input(shape=input_shape)
    encoded = Dense(128, activation='relu')(inputs)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    
    autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    autoencoder.compile(optimizer=Adam(), loss='mse')
    return autoencoder, encoder

# Main Function
def main():
    eeg_file = '/kaggle/input/mental-state/mental-state.csv'
    
    print("\nLoading EEG Dataset...")
    eeg_df = pd.read_csv(eeg_file)
    X, y = preprocess_data(eeg_df)

    class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
    class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

    X_train, X_val = X[:int(0.8 * len(X))], X[int(0.8 * len(X)):]
    y_train, y_val = y[:int(0.8 * len(y))], y[int(0.8 * len(y))]

    autoencoder, encoder = build_autoencoder((X_train.shape[1],))
    autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_val, X_val), verbose=1)

    X_train_encoded = encoder.predict(X_train)
    X_val_encoded = encoder.predict(X_val)

    input_encoded = Input(shape=(32,))
    classifier_output = Dense(len(np.unique(y)), activation='softmax')(input_encoded)
    classifier_model = Model(input_encoded, classifier_output)
    classifier_model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    classifier_model.fit(X_train_encoded, y_train, epochs=10, batch_size=32, validation_data=(X_val_encoded, y_val), verbose=1)

    y_pred = np.argmax(classifier_model.predict(X_val_encoded), axis=1)
    accuracy = accuracy_score(y_val, y_pred)
    print(f"\nFinal Model Accuracy: {accuracy:.4f}")
    print(classification_report(y_val, y_pred))

if __name__ == "__main__":
    main()



Loading EEG Dataset...
Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.1701 - val_loss: 0.0014
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0011 - val_loss: 0.0012
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Epoch 1/10
52/62 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - ac

IndexError: tuple index out of range

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import gc
from scipy.fft import dct, idct
from scipy.stats import skew, kurtosis, entropy
from sklearn.utils.class_weight import compute_class_weight

# Fourier Decomposition Method (FDM)
def fdm(X, fs, fc):
    N = X.shape[0]
    fc = np.sort(fc)

    if fc[0] != 0:
        fc = np.hstack((0, fc))
    if fc[-1] != fs / 2:
        fc = np.hstack((fc, fs / 2))

    dct_type = 2
    K = np.round(2 * N * fc / fs).astype(int)
    Hk = np.zeros((N, 1, len(K) - 1))

    for i in range(len(K) - 1):
        Hk[K[i]:K[i + 1], :, i] = 1

    Xk = dct(X, type=dct_type, n=N, axis=0, norm='ortho')
    Yk = np.einsum('ij,ijk->ijk', Xk, Hk)
    Y = idct(Yk, type=dct_type, n=N, axis=0, norm='ortho')

    return np.squeeze(Y) if X.ndim == 1 else Y

# Feature Extraction
def extract_features(X):
    mean_amplitude = np.mean(X, axis=1, keepdims=True)
    variance = np.var(X, axis=1, keepdims=True)
    skewness = skew(X, axis=1, keepdims=True)
    kurt = kurtosis(X, axis=1, keepdims=True)
    
    X_normalized = np.abs(X) / np.sum(np.abs(X), axis=1, keepdims=True)
    signal_entropy = np.apply_along_axis(entropy, axis=1, arr=X_normalized).reshape(-1, 1)
    
    return np.hstack([mean_amplitude, variance, skewness, kurt, signal_entropy])

# Preprocess Data
def preprocess_data(df, label_column='Label', normalize=True):
    labels = df[label_column].values
    features = df.drop(columns=[label_column]).values
    features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
    additional_features = extract_features(features)
    features = np.hstack([features, additional_features])
    if normalize:
        features = features / np.max(np.abs(features), axis=1, keepdims=True)
    return features, labels

# Autoencoder Model
def build_autoencoder(input_shape):
    inputs = Input(shape=input_shape)
    encoded = Dense(128, activation='relu')(inputs)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    
    autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    autoencoder.compile(optimizer=Adam(), loss='mse')
    return autoencoder, encoder

# Main Function
def main():
    eeg_file = '/kaggle/input/mental-state/mental-state.csv'
    
    print("\nLoading EEG Dataset...")
    eeg_df = pd.read_csv(eeg_file)
    X, y = preprocess_data(eeg_df)

    class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
    class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

    X_train, X_val = X[:int(0.8 * len(X))], X[int(0.8 * len(X)):]
    y_train, y_val = y[:int(0.8 * len(y))], y[int(0.8 * len(y)):]

    autoencoder, encoder = build_autoencoder((X_train.shape[1],))
    autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_val, X_val), verbose=1)

    X_train_encoded = encoder.predict(X_train)
    X_val_encoded = encoder.predict(X_val)

    input_encoded = Input(shape=(X_train_encoded.shape[1],))
    classifier_output = Dense(len(np.unique(y)), activation='softmax')(input_encoded)
    classifier_model = Model(input_encoded, classifier_output)
    classifier_model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = classifier_model.fit(X_train_encoded, y_train, epochs=50, batch_size=50, validation_data=(X_val_encoded, y_val), verbose=1)
    
    for epoch, acc in enumerate(history.history['accuracy'], 1):
        print(f"Epoch {epoch}: Training Accuracy = {acc:.4f}")
    
    for epoch, val_acc in enumerate(history.history['val_accuracy'], 1):
        print(f"Epoch {epoch}: Validation Accuracy = {val_acc:.4f}")
    
    y_pred = np.argmax(classifier_model.predict(X_val_encoded), axis=1)
    accuracy = accuracy_score(y_val, y_pred)
    print(f"\nFinal Model Accuracy: {accuracy:.4f}")
    print(classification_report(y_val, y_pred))

if __name__ == "__main__":
    main()



Loading EEG Dataset...
Epoch 1/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.1658 - val_loss: 0.0014
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 9/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 10/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 11/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 12/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from scipy.stats import skew, kurtosis, entropy

# Feature Extraction
def extract_features(X):
    mean_amplitude = np.mean(X, axis=1, keepdims=True)
    variance = np.var(X, axis=1, keepdims=True)
    skewness = skew(X, axis=1, keepdims=True)
    kurt = kurtosis(X, axis=1, keepdims=True)
    
    X_normalized = np.abs(X) / np.sum(np.abs(X), axis=1, keepdims=True)
    signal_entropy = np.apply_along_axis(entropy, axis=1, arr=X_normalized).reshape(-1, 1)
    
    return np.hstack([mean_amplitude, variance, skewness, kurt, signal_entropy])

# Preprocess Data
def preprocess_data(df, label_column='Label', normalize=True):
    labels = df[label_column].values
    features = df.drop(columns=[label_column]).values
    features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
    additional_features = extract_features(features)
    features = np.hstack([features, additional_features])
    if normalize:
        features = features / np.max(np.abs(features), axis=1, keepdims=True)
    return features, labels

# Autoencoder Model
def build_autoencoder(input_shape):
    inputs = Input(shape=input_shape)
    encoded = Dense(128, activation='relu')(inputs)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    
    autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    autoencoder.compile(optimizer=Adam(), loss='mse')
    return autoencoder, encoder

# Load Dataset
eeg_file = '/kaggle/input/mental-state/mental-state.csv'
eeg_df = pd.read_csv(eeg_file)
X, y = preprocess_data(eeg_df)

# Split Dataset
split_idx = int(0.8 * len(X))
X_train, X_val = X[:split_idx], X[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]

# Train Autoencoder
autoencoder, encoder = build_autoencoder((X_train.shape[1],))
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_val, X_val), verbose=1)

# Encode Features
X_train_encoded = encoder.predict(X_train)
X_val_encoded = encoder.predict(X_val)

# Train Random Forest and Extra Trees Classifiers
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
et_model = ExtraTreesClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train_encoded, y_train)
et_model.fit(X_train_encoded, y_train)

# Ensemble Predictions using Soft Voting
rf_pred_proba = rf_model.predict_proba(X_val_encoded)
et_pred_proba = et_model.predict_proba(X_val_encoded)
final_pred_proba = (rf_pred_proba + et_pred_proba) / 2
final_pred = np.argmax(final_pred_proba, axis=1)

# Evaluate Model
accuracy = accuracy_score(y_val, final_pred)
print(f"\nEnsemble Model Accuracy: {accuracy:.4f}")
print(classification_report(y_val, final_pred))


Epoch 1/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.1784 - val_loss: 0.0014
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 9/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 10/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 11/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 12/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0011 - val_l

In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin

# Feature Extraction
def extract_features(X):
    from scipy.stats import skew, kurtosis, entropy
    mean_amplitude = np.mean(X, axis=1, keepdims=True)
    variance = np.var(X, axis=1, keepdims=True)
    skewness = skew(X, axis=1, keepdims=True)
    kurt = kurtosis(X, axis=1, keepdims=True)
    
    X_normalized = np.abs(X) / np.sum(np.abs(X), axis=1, keepdims=True)
    signal_entropy = np.apply_along_axis(entropy, axis=1, arr=X_normalized).reshape(-1, 1)
    
    return np.hstack([mean_amplitude, variance, skewness, kurt, signal_entropy])

# Preprocess Data
def preprocess_data(df, label_column='Label', normalize=True):
    labels = df[label_column].values
    features = df.drop(columns=[label_column]).values
    features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
    additional_features = extract_features(features)
    features = np.hstack([features, additional_features])
    if normalize:
        features = features / np.max(np.abs(features), axis=1, keepdims=True)
    return features, labels

# Autoencoder Model
def build_autoencoder(input_shape):
    inputs = Input(shape=input_shape)
    encoded = Dense(256, activation=LeakyReLU())(inputs)
    encoded = Dense(128, activation=LeakyReLU())(encoded)
    encoded = Dense(64, activation=LeakyReLU())(encoded)
    encoded = Dense(32, activation=LeakyReLU())(encoded)
    
    decoded = Dense(64, activation=LeakyReLU())(encoded)
    decoded = Dense(128, activation=LeakyReLU())(decoded)
    decoded = Dense(256, activation=LeakyReLU())(decoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    
    autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return autoencoder, encoder

# Load Dataset
eeg_file = '/kaggle/input/mental-state/mental-state.csv'
eeg_df = pd.read_csv(eeg_file)
X, y = preprocess_data(eeg_df)

# Apply SMOTE for class balancing
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Split Dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Autoencoder
autoencoder, encoder = build_autoencoder((X_train.shape[1],))
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_val, X_val), verbose=1)

# Encode Features
X_train_encoded = encoder.predict(X_train)
X_val_encoded = encoder.predict(X_val)

# Apply PCA for Feature Reduction
pca = PCA(n_components=20)  # Keeping top 20 components
X_train_pca = pca.fit_transform(X_train_encoded)
X_val_pca = pca.transform(X_val_encoded)

# Train Random Forest and Extra Trees Classifiers with Hyperparameter Tuning
rf_model = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42)
et_model = ExtraTreesClassifier(n_estimators=200, max_depth=20, random_state=42)

rf_model.fit(X_train_pca, y_train)
et_model.fit(X_train_pca, y_train)

# Define a Custom Stacking Model
class StackingModel(BaseEstimator, ClassifierMixin):
    def __init__(self, base_models, meta_model):
        self.base_models = base_models
        self.meta_model = meta_model

    def fit(self, X, y):
        self.base_models_ = [model.fit(X, y) for model in self.base_models]
        meta_features = np.column_stack([model.predict(X) for model in self.base_models_])
        self.meta_model_ = self.meta_model.fit(meta_features, y)
        return self

    def predict(self, X):
        meta_features = np.column_stack([model.predict(X) for model in self.base_models_])
        return self.meta_model_.predict(meta_features)

# Train the Stacking Model
meta_model = XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=5, random_state=42)
stacking_model = StackingModel(base_models=[rf_model, et_model], meta_model=meta_model)
stacking_model.fit(X_train_pca, y_train)

# Evaluate the Model
y_pred = stacking_model.predict(X_val_pca)
accuracy = accuracy_score(y_val, y_pred)
print(f"\nStacked Ensemble Model Accuracy: {accuracy:.4f}")
print(classification_report(y_val, y_pred))


Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.1253 - val_loss: 0.0015
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0016 - val_loss: 0.0015
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0016 - val_loss: 0.0015
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0016 - val_loss: 0.0015
Epoch 9/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 10/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 11/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0021 - val_loss: 0.0018
Epoch 12/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0020 - val_l

In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin

# Load Dataset
eeg_file = '/kaggle/input/mental-state/mental-state.csv'
eeg_df = pd.read_csv(eeg_file)

# Preprocess Data
def preprocess_data(df, label_column='Label'):
    labels = df[label_column].values
    features = df.drop(columns=[label_column]).values
    features = np.nan_to_num(features)
    return features, labels

X, y = preprocess_data(eeg_df)

# Apply SMOTE for class balancing
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Split Dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Autoencoder Model with Fine-Tuning
def build_autoencoder(input_shape):
    inputs = Input(shape=input_shape)
    encoded = Dense(512, activation=LeakyReLU())(inputs)
    encoded = BatchNormalization()(encoded)
    encoded = Dense(256, activation=LeakyReLU())(encoded)
    encoded = Dropout(0.2)(encoded)
    encoded = Dense(128, activation=LeakyReLU())(encoded)
    
    decoded = Dense(256, activation=LeakyReLU())(encoded)
    decoded = Dense(512, activation=LeakyReLU())(decoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    
    autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    autoencoder.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')
    return autoencoder, encoder

autoencoder, encoder = build_autoencoder((X_train.shape[1],))
autoencoder.fit(X_train, X_train, epochs=100, batch_size=64, validation_data=(X_val, X_val), verbose=1)

# Encode Features
X_train_encoded = encoder.predict(X_train)
X_val_encoded = encoder.predict(X_val)

# Apply PCA
pca = PCA(n_components=30)
X_train_pca = pca.fit_transform(X_train_encoded)
X_val_pca = pca.transform(X_val_encoded)

# Feature Selection using RFE
selector = RFE(estimator=RandomForestClassifier(n_estimators=50), n_features_to_select=20)
X_train_selected = selector.fit_transform(X_train_pca, y_train)
X_val_selected = selector.transform(X_val_pca)

# Hyperparameter Tuning for Base Models
rf_params = {'n_estimators': [100, 200, 300], 'max_depth': [10, 20, 30]}
et_params = {'n_estimators': [100, 200, 300], 'max_depth': [10, 20, 30]}

rf_model = RandomizedSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=3, n_iter=5, n_jobs=-1)
et_model = RandomizedSearchCV(ExtraTreesClassifier(random_state=42), et_params, cv=3, n_iter=5, n_jobs=-1)

rf_model.fit(X_train_selected, y_train)
et_model.fit(X_train_selected, y_train)

# Stacking Model with LightGBM
class StackingModel(BaseEstimator, ClassifierMixin):
    def __init__(self, base_models, meta_model):
        self.base_models = base_models
        self.meta_model = meta_model

    def fit(self, X, y):
        self.base_models_ = [model.best_estimator_.fit(X, y) for model in self.base_models]
        meta_features = np.column_stack([model.predict(X) for model in self.base_models_])
        self.meta_model.fit(meta_features, y)
        return self

    def predict(self, X):
        meta_features = np.column_stack([model.predict(X) for model in self.base_models_])
        return self.meta_model.predict(meta_features)

meta_model = LGBMClassifier(n_estimators=200, learning_rate=0.05, max_depth=5, scale_pos_weight=1.2, random_state=42)
stacking_model = StackingModel(base_models=[rf_model, et_model], meta_model=meta_model)
stacking_model.fit(X_train_selected, y_train)

# Evaluate Model
y_pred = stacking_model.predict(X_val_selected)
accuracy = accuracy_score(y_val, y_pred)
print(f"\nOptimized Stacked Ensemble Model Accuracy: {accuracy:.4f}")
print(classification_report(y_val, y_pred))


Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 9632535.0000 - val_loss: 11118223.0000
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 9840516.0000 - val_loss: 11118223.0000
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 9343451.0000 - val_loss: 11118222.0000
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 8092721.5000 - val_loss: 11118221.0000
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 9285255.0000 - val_loss: 11118220.0000
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 7951411.5000 - val_loss: 11118220.0000
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 9091162.0000 - val_loss: 11118220.0000
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 10032054.0000 - val_loss: 11118220.0000
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 9650591.0000 - val_loss: 11118220.0000
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 9036493.0000 - val_loss: 11118220.0000
Epoch 11